# Google Docs Comment Exporter

This script transfers comments from a set of Google documents into a CSV file.

## Setup

1. Create a Google Cloud Platform account at [console.cloud.google.com](console.cloud.google.com). I've had problems using my CMU account so I use a personal Gmail account.
2. Create a project in GCP, then select it to work within that project.
3. Go to APIs and Services > Library in the sidebar, find **Google Drive API**, and enable it for the GCP project. (You shouldn't need to worry about payment - this use case shouldn't use anywhere close to the free tier maximum.)
4. Go to APIs and Services > Credentials. Click Create Credentials, then OAuth Client ID. For Application Type, choose Desktop App. Once the client is created, copy the client ID into the client_id variable below, and click Download JSON to save the client secret. Input the client secret path into the client_secret variable below.
5. Install the Python Google Drive API: `pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib`

In [ ]:
import os
import sys
import requests
import html
import re
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

In [ ]:
client_id = "ABCDEFG.apps.googleusercontent.com" # Client ID from GCP console
client_secret = "client_secret.json" # Path to client secret JSON file downloaded from GCP console, relative to this notebook

## Initializing Google Drive API

Running the following cell should open an OAuth window to log you in. You should log in to the account that has the files you need to read from Google Drive. If the authentication worked, you should see it print a list of some files in your Drive.

In [ ]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly',
         'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.readonly',
        'https://www.googleapis.com/auth/drive.file']

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(client_secret, SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('drive', 'v3', credentials=creds)

# Call the Drive v3 API
results = service.files().list(
    pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))

## Reading individual file comments

To test how the comment structure works, try the following cell. You'll need to fill in a file ID from your Drive (it's in the document's URL, like `https://docs.google.com/document/d/<FILE_ID>/edit?usp=sharing`).

In [ ]:
file_id = "" # something like "15dR04q_whT11lHKDSsSVFkYsxoC8_wKCDUb0eh9vyvc"
result = service.comments().list(fileId=file_id, fields="*").execute()
for comment in result['comments']:
    if comment['resolved']: continue
    print(comment['author']['displayName'], comment['anchor'])
    print(comment)
    print("")

## Finding all files in a folder

The following code builds a list of files from a Google Drive directory. We will extract comments from the files stored in the `files` list.

To populate the `parent_folder` variable, get the folder ID by opening the folder in Google Drive and then copy the ID in the URL bar. For example, the URL might look like `https://drive.google.com/drive/u/0/folders/<folder ID>`.

In [ ]:
parent_folder = "" # Folder ID (in the URL when you view that folder in Drive)

# The q parameter below (query) can be modified to further limit the results. For example, we used:
# f"'{parent_folder}' in parents and mimeType = 'application/vnd.google-apps.document' and (name contains 'Interview' or name contains 'Inquiry')"
parent_result = service.files().list(q=f"'{parent_folder}' in parents").execute()
files = []

for file in sorted(parent_result['files'], key=lambda x: x['name']):
    # Optionally filter file names by regex here
#     if not re.search(r'^(C|S)\d+ (Interview|Inquiry)', file['name']) or 'old' in file['name']:
#         continue
    print(file['name'], file['id'])
    files.append(file)

## Writing out the comments

The following code iterates over the list of `files` built above, and requests comments for each document iteratively (because Drive comments are returned in pages if there are too many). It writes the resulting comments dataframe to CSV at the specified `out_path`.

In [ ]:
out_path = "comments.csv"

In [ ]:
data = []
for file in files:
    print(file['name'])
    token = 'first'
    while token:
        result = service.comments().list(fileId=file['id'], fields="*", pageToken=token if token != 'first' else None).execute()

        for comment in result['comments']:
            # This code filters out anything that is resolved, deleted, or where the comment begins with MEMO.
            # We used MEMO to specify when a comment on the doc was not supposed to be included in the qualitative analysis.
            if comment.get('resolved', False) or comment.get('deleted', False) or comment['content'].startswith("MEMO"): continue
            tag = re.search(r"@(\S+)", comment['htmlContent'])
            has_replies = any(r['content'] != '+1' and not r['content'].startswith("MEMO") for r in comment.get('replies', []))
            
            data.append({
                "Summary": "{} ({}, {})".format(
                    html.unescape(comment['content']),
                    file['name'],
                    comment['id']),
                "Attn (Replies other than +1 or Tag)": has_replies or tag is not None,
                "Author": comment['author']['displayName'],
                "Source": file['name'],
                "Link": "https://docs.google.com/document/d/{}?disco={}".format(file['id'], comment['id']),
                "Quote": html.unescape(comment['quotedFileContent']['value'] if 'quotedFileContent' in comment else ''),
                "Comment": html.unescape(comment['content']),
                "Replies": str(len(comment.get('replies', []))),
                "Tag": tag.group(1) if tag is not None else '',
            })
            
        token = result.get('nextPageToken', None)

pd.DataFrame(data).to_csv(out_path)
print("Done.")